# Análisis de las columnas numéricas

In [5]:
# Importaciones de paquetes
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x : '%.4f' % x)

# Importaciones de unidades de soporte
import sys
sys.path.append('..')
from src import sp_nulos_cat as sp

In [6]:
df = pd.read_excel('../data/datos_churn_limpios.xlsx')

In [7]:
# Crear un dataframe solo con las columnas numericas
df_num = df.select_dtypes(include= np.number).columns.to_list()

## Gestión de valores atípicos

Antes de gestionar los valores nulos, trabajaremos los valores atípicos, ya que  pueden distorsionar la media y la mediana, afectando la correcta imputación de los valores nulos. Además, eliminarlos primero mejora la calidad de las visualizaciones y evita sesgos en los modelos predictivos.

In [ ]:
# Observar las medidas estadísticas
df[df_num].describe().T

In [ ]:
# Comprobamos la presencia de valores atípicos comparando los histogramas con los diagramas de cajas de cada columna
sp.subplot_col_num(df, df_num)

In [ ]:
# Detectamos valores atípicos en cinco columnas. Vamos a calcular su porcentaje en cada columna para decidir que hacer con ellos.
dicc_outliers = {'avg_monthly_gb_download':23,
                 'extra_data_charges':1,
                 'total_charges':4000 ,
                 'average_monthly_expenses':80} 
for col, out in dicc_outliers.items():
  outliers = df[col][df[col] > out].count()
  print(f'Para la columna {col.upper()} tenemos {outliers}, lo que representa un {round(outliers/df.shape[0] *100,3)}% de los datos')

Aunque algunas variables presentan valores atípicos, su porcentaje es bajo y tienen sentido en el contexto del análisis. Modificarlos podría eliminar información valiosa sobre clientes con patrones de consumo específicos, por lo que decidimos dejarlos tal cual.

## Gestión de nulos de las columnas numéricas

In [ ]:
col_con_nulos = df.columns[df.isnull().any()]
columnas_nulos_info = pd.DataFrame({
        "columna": col_con_nulos,
        "NumeroNulos": [df[col].isnull().sum() for col in col_con_nulos],
        "PorentajeNulos": [(df[col].isnull().sum()/df.shape[0])*100 for col in col_con_nulos]})
display(columnas_nulos_info)

`total_charges` tenía un porcentaje muy pequeño de valores nulos (0.2%). Por un lado, rellenamos con 0 para los clientes que tienen 'tenure = 0', porque aún no han acumulado cargos. Para los demás casos, usamos la mediana en lugar del promedio, ya que nos ayuda a evitar que se distorsionen los datos.

In [ ]:
# Imputar nulos en 'total_charges'
df.loc[df['customer_tenure_(in_months)']==0, 'total_charges'] = 0 # Imputar 'total_charges' con 0 para clientes con 'tenure = 0'
median_total_charges = df['total_charges'].median() 
df['total_charges'].fillna(median_total_charges, inplace=True) # Imputar los valores nulos restantes con la mediana

`monthly_charge` tenía un porcentaje muy pequeño de valores nulos (0.2%), por lo que se rellenaron con la mediana, ya que no tenía una relación significativa con otras variables. Esto garantiza la coherencia en los datos sin introducir sesgos ni afectar significativamente el análisis.

In [ ]:
# Imputar nulos en 'monthly_charge' con la mediana
median_monthly_charge = df['monthly_charge'].median()
df['monthly_charge'].fillna(median_monthly_charge, inplace=True)

In [ ]:
# Comprobar que no hay nulos en ninguna columna
df.isnull().sum()

In [22]:
# Guardar el archivo
df.to_excel("../data/datos_churn_limpios.xlsx", index=False)